# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
weather_dataframe_csv = "output_data/weather_dataframe.csv"
weather_df = pd.read_csv(weather_dataframe_csv) 
weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,atuona,5,PF,1603225116,75,-9.80,-139.03,77.54,19.44
1,1,port moresby,100,CD,1603225116,62,-7.03,29.76,78.15,7.65
2,2,avarua,36,CK,1603225116,88,-21.21,-159.78,77.00,5.82
3,3,hithadhoo,98,MV,1603225116,80,-0.60,73.08,82.31,14.12
4,4,san nicolas,0,ML,1603224903,29,13.30,-4.90,86.81,1.14
...,...,...,...,...,...,...,...,...,...,...
323,323,qaqortoq,40,GL,1603225146,86,60.72,-46.03,33.80,3.36
324,324,kushmurun,96,KZ,1603225146,54,52.45,64.63,50.38,5.39
325,325,grand river south east,1,US,1603225146,13,38.88,-109.50,78.01,5.82
326,326,evensk,80,RU,1603225147,88,61.95,159.23,34.34,5.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
humidity = weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["Lat", "Lng"]]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=True, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
narrowed_city_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head(10)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,7,singaraja,0,ID,1603225117,64,-8.11,115.09,79.38,3.22
1,10,mtambile,0,TZ,1603225117,81,-5.38,39.70,78.19,10.27
2,28,rikitea,0,PF,1603225118,78,-23.12,-134.97,73.45,14.81
3,57,cape town,0,CL,1603225121,31,-36.41,-71.90,75.20,13.87
4,58,menongue,0,AO,1603225121,19,-14.66,17.69,73.51,3.42
5,172,najran,0,SA,1603225131,32,17.49,44.13,71.60,4.70
6,200,jalu,0,LY,1603225134,60,29.03,21.55,71.65,14.23
7,213,muscat,0,OM,1603224882,60,23.61,58.59,73.40,2.24
8,263,nhulunbuy,0,AU,1603225140,88,-12.23,136.77,73.40,3.36
9,270,tura,0,IN,1603225141,92,25.52,90.22,72.30,3.18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
hotel_df = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrowed_city_df["Hotel Name"] = hotel_df
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head(10)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,singaraja,0,ID,1603225117,64,-8.11,115.09,79.38,3.22,Singaraja
1,10,mtambile,0,TZ,1603225117,81,-5.38,39.70,78.19,10.27,Mkoani
2,28,rikitea,0,PF,1603225118,78,-23.12,-134.97,73.45,14.81,Rikitea
3,57,cape town,0,CL,1603225121,31,-36.41,-71.90,75.20,13.87,San Carlos
4,58,menongue,0,AO,1603225121,19,-14.66,17.69,73.51,3.42,Menongue
5,172,najran,0,SA,1603225131,32,17.49,44.13,71.60,4.70,Najran
6,200,jalu,0,LY,1603225134,60,29.03,21.55,71.65,14.23,Jalu
7,213,muscat,0,OM,1603224882,60,23.61,58.59,73.40,2.24,Muscat
8,263,nhulunbuy,0,AU,1603225140,88,-12.23,136.77,73.40,3.36,Nhulunbuy
9,270,tura,0,IN,1603225141,92,25.52,90.22,72.30,3.18,Tura


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [61]:
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))